Step 1: build a surface water network. You can "pickle" this, so it doesn't need to be repeated.

n = swn.SurfaceWaterNetwork.from_lines(gdf.geometry)

n.to_pickle("surface-water-network.pkl")

# then in a later session, skip the above and just do:

n = swn.SurfaceWaterNetwork.from_pickle("surface-water-network.pkl")

Step 2: load a MF6 model, then find the intersections:

sim = flopy.mf6.MFSimulation.load(...)

m = sim.get_model(...)

nm = swn.SwnMf6.from_swn_flopy(n, m)

Most of the results will be in the nm.reaches property, but other reach datasets will need to be specified, including "man", "rbth", "rgrd", "rhk", "rtp", and "rwid". There are two methods to format the PACKAGEDATA:

nm.flopy_packagedata

nm.write_packagedata("packagedata.dat")

Similar with CONNECTIONDATA:

nm.flopy_connectiondata

nm.write_connectiondata("connectiondata.dat")

There are a few "helper" methods to sort out things like "set_reach_slope" based on a few methods. One missing one is "set_reach_elevation" or whatever to make the reaches fit in the layer and/or move the layer elevations to fit the stream.

And lastly, there is no PERIOD data yet. I'm working on this, which is holding up the merge.


In [1]:
import geopandas
import os
import swn
import flopy
import numpy as np
import time

# SW network pickle

#only do this once... takes forever... then load pickle
gdb_dir = 'D:\modelling\data'
gdb_fname = 'nzRec2_v5.gdb'
gdb_path = os.path.join(gdb_dir, gdb_fname)
# Read national data of streams
gdf_lines = geopandas.read_file(gdb_path, layer='riverlines')
gdf_lines.set_index('nzsegment', inplace=True, verify_integrity=True)
gdf_ws = geopandas.read_file(gdb_path, layer='rec2ws')
gdf_ws.set_index('nzsegment', inplace=True, verify_integrity=True)
# Convert MultiLineString -> LineString
lines = gdf_lines.geometry.apply(lambda x: x.geoms[0]) #what is geoms[0]
polygons = gdf_ws.geometry.apply(lambda x: x.geoms[0])
#ni_lines = gdf_lines.loc[gdf_lines.index < 10000000, "geometry"]
# requires reindex otherwise failure in core.from_lines
t0=time.time()
n = swn.SurfaceWaterNetwork.from_lines(lines,polygons.reindex(index=lines.index))
print(time.time()-t0)
n.to_pickle("surface-water-network.pkl")

# get the pickle

In [2]:
n = swn.SurfaceWaterNetwork.from_pickle("surface-water-network.pkl")

# Load MF6 model

In [3]:
os.getcwd()

'D:\\modelling\\surface-water-network'

In [4]:
sim_ws=os.path.join('..','zmodels','20210622_simulation','wairau_240_3')
model_name='wairau_240_3'
sim=flopy.mf6.MFSimulation.load(sim_ws=sim_ws)
gwf=sim.get_model(model_name)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package ghb...
    loading package rch...
    loading package drn...
    loading package npf...
    loading package sto...
    loading package oc...
  loading ims package wairau_240_3...


## spatial reference for model

In [5]:
#sr=flopy.utils.reference.SpatialReference.from_gridspec(os.path.join(sim_ws,model_name+'.grid.spc'))
#gwf.dis.xorigin=sr.xul
#gwf.dis.yorigin=sr.yul-np.sum(gwf.dis.delr.data)
#gwf.dis.write()

In [6]:
type(gwf)

flopy.mf6.mfmodel.MFModel

In [7]:
# this also takes forever
t0=time.time()
ngwf = swn.SwnMf6.from_swn_flopy(n, gwf,reach_include_fraction=0)
print(time.time()-t0)

370.4846336841583


In [8]:
# https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-sfr.html?highlight=ustrf#block-packagedata
# started from sagehen example, tweaked
# can do ngwf.default_packagedata() now?
d={'rwid':10.0,'man':0.04,'ustrf':1.0,'ndv': 0}
for k in ["man", "rwid"]:
    ngwf.reaches[k]=d[k]

# was taking much time to fail with no zcoord, now takes 1.2 sec
t0=time.time()
#zcoord_ab or grid_top
try:
    ngwf.set_reach_slope(method='zcoord_ab')
except:
    print(time.time()-t0)


In [9]:
# check for nans, should have been fixed, taken care of
ngwf.set_reach_slope(method='top_len')

17:27:29.814:WARNING:SwnMf6:enforcing min_slope for 201 reaches (0.50%)


In [10]:
ngwf.reaches

,geometry,segnum,segndist,k,i,j,iseg,ireach,rlen,to_rno,from_rnos,to_div,ustrf,man,rwid,min_slope,rgrd
rno,,,,,,,,,,,,,,,,,
1,"LINESTRING (1589889.333 5333340.791, 1589920.0...",13058095,0.004261,0,396,55,1,1,30.666800,2,{},0,1.0,0.04,10.0,0.001,7.931353
2,"LINESTRING (1589920.000 5333340.786, 1590054.2...",13058095,0.043592,0,396,56,1,2,252.418213,3,{1},0,1.0,0.04,10.0,0.001,0.907762
3,"LINESTRING (1590160.000 5333370.745, 1590294.1...",13058095,0.113735,0,396,57,1,3,252.426113,4,{2},0,1.0,0.04,10.0,0.001,0.684987
4,"LINESTRING (1590400.000 5333340.723, 1590414.1...",13058095,0.187329,0,396,58,1,4,277.254636,5,{3},0,1.0,0.04,10.0,0.001,0.433135
5,"LINESTRING (1590640.000 5333430.661, 1590654.0...",13058095,0.262646,0,396,59,1,5,264.836425,6,{4},0,1.0,0.04,10.0,0.001,0.366318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40367,"LINESTRING (1687260.565 5405040.000, 1687263.4...",11024563,0.421101,0,98,461,9510,2,113.039624,40368,{40366},0,1.0,0.04,10.0,0.001,0.003925
40368,"LINESTRING (1687360.000 5405007.154, 1687383.3...",11024563,0.767288,0,98,462,9510,3,231.819660,40369,{40367},0,1.0,0.04,10.0,0.001,0.001000
40369,"LINESTRING (1687548.326 5404902.159, 1687600.0...",11100009,0.045376,0,98,462,9511,1,55.096243,40370,{40368},0,1.0,0.04,10.0,0.001,0.001483


In [15]:
# no rno=0 so missing labels error, floats because of nans
top=gwf.dis.top.array
botm=gwf.dis.botm.array

In [ ]:
mindz=0.001
loop=0
df=ngwf.reaches.copy()
rdf=ngwf.reaches.copy()
cont=True
while cont:   
    loop=loop+1
    chg=0
    for idx,r in df.iterrows(): 
        ij=(int(r['i']),int(r['j']))
        trno=int(r['to_rno'])
        rdf.loc[idx,'rtp']=top[ij]
        rdic={}
        if trno!=0:
            #print(idx,rdf.loc[idx,'rtp'],top[i,j],[b[i,j] for b in botm])    
            titj=(int(df.loc[trno,'i']),int(df.loc[trno,'j']))
            # build list of props moving downstream
            while trno!=0 and top[titj]>=top[ij]-mindz:
                dz=top[titj]-(top[ij]-mindz)
                if titj not in rdic.keys():
                    rdic[titj]={'nz':[dz],'rno':[trno]}
                else:
                    rdic[titj]['nz'].append(dz)
                    rdic[titj]['rno'].append(trno)
                # set next reach
                trno=df.loc[trno,'to_rno']
                if trno!=0:
                    titj=(int(df.loc[trno,'i']),int(df.loc[trno,'j']))
                
            # now adjust each in list
            if len(rdic.keys())>0:
                for titj in rdic.keys():
                    dz=np.max(rdic[titj]['nz'])
                    top[titj]=np.max([top[titj]-dz,mindz])
                    for rno in rdic[titj]['rno']:
                        rdf.loc[rno,'rtp']=top[titj]
                    for b in range(0,botm.shape[0]):
                        botm[b][titj]=botm[b][titj]-dz
        chg=chg+len(rdic.keys())
    if chg==0:
        cont=False
    else:
        print('{} changed in loop {}'.format(chg,loop))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(top)
plt.colorbar()

In [ ]:
plt.imshow(gwf.dis.top.array-top)
plt.colorbar()

In [ ]:
rdf.loc[:,'rbth']=2

In [ ]:
f=gwf.dis.top.get_file_entry().split("'")[1]
fpth=os.path.join(sim_ws+'_sfr',f)
np.savetxt(f,top)

In [ ]:
for b in range(0,botm.shape[0]):
    f=gwf.dis.botm[b].fname
    fpth=os.path.join(sim_ws+'_sfr',f)
    np.savetxt(f,botm[b])

In [ ]:
ngwf.reaches=rdf

In [ ]:
ngwf.set_reach_data_from_array('rhk',gwf.npf.k.array[0])

In [ ]:
n.segments.to_csv(os.path.join(sim_ws+'_sfr',model_name+'_znseg_mf6rch.csv'))

In [ ]:
mask=[s for s in n.segments.index if (len(n.segments.loc[s,'from_segnums'])==0) & (n.segments.loc[s,'to_segnum']==0)]

In [ ]:
len(mask)

In [ ]:
n.segments.drop(mask,inplace=True)

In [ ]:


# There are two methods to format the PACKAGEDATA:
ngwf.flopy_packagedata
ngwf.write_packagedata(os.path.join(sim_ws+'_sfr',model_name+'.sfr0.reach.dat'))

#Similar with CONNECTIONDATA:
ngwf.flopy_connectiondata
ngwf.write_connectiondata(os.path.join(sim_ws+'_sfr',model_name+'.sfr0.connection.dat'))


In [ ]:
sfr=flopy.mf6.ModflowGwfsfr(gwf,packagedata={'filename':model_name+'.sfr0.reach.dat'},
                            connectiondata={'filename':model_name+'.sfr0.connection.dat'},
                           nreaches=len(ngwf.reaches),budget_filerecord=model_name + "_sfr.bud",
                            save_flows=True)
#gwf.register_package(sfr)
sfr.write()

In [ ]:
model_name

In [ ]:
gwf.write()

In [ ]:
help(ngwf)

# write shapefile, but not sfr info from grid intersection

In [ ]:
swn.file.gdf_to_shapefile(n.segments, 'segments.shp')